# 01 — Parse PDON & PMDO ontologies (Colab)

This notebook loads **PDON** and **PMDO** from local files, validates parsing, extracts basic statistics (classes/properties), and optionally exports Turtle versions for easier downstream work.

**Inputs** (place in `ontologies/`):
- `pdon.xrdf`
- `pmdo.xrdf`

**Outputs** (written to `output/`):
- `pdon.ttl` (optional)
- `pmdo.ttl` (optional)
- `ontologies_summary.csv`


## 0) Install dependencies

In [ ]:
!pip -q install rdflib pandas owlrl

## 1) Project folders

In Colab, we work under `/content`. If you cloned the GitHub repo, set `PROJECT_DIR` accordingly.
Otherwise, this notebook will create a minimal folder structure.


In [ ]:
from pathlib import Path

# If you cloned the repo, set PROJECT_DIR = Path('/content/ppmi-ontology-alignment')
PROJECT_DIR = Path('/content/ppmi-ontology-alignment')

# Fallback (no clone): use /content and create project-like folders
if not PROJECT_DIR.exists():
    PROJECT_DIR = Path('/content/ppmi-ontology-alignment')
    PROJECT_DIR.mkdir(parents=True, exist_ok=True)

ONTO = PROJECT_DIR / 'ontologies'
DATA = PROJECT_DIR / 'data'
OUT  = PROJECT_DIR / 'output'

for p in [ONTO, DATA, OUT]:
    p.mkdir(parents=True, exist_ok=True)

PROJECT_DIR, ONTO, DATA, OUT


## 2) Upload files (if needed)

If `pdon.xrdf` and `pmdo.xrdf` are already in `ontologies/`, you can skip this step.


In [ ]:
from google.colab import files
import shutil
from pathlib import Path

required = ['pdon.xrdf', 'pmdo.xrdf']

missing = [f for f in required if not (ONTO / f).exists()]
print("Missing:", missing)

if missing:
    uploaded = files.upload()
    for fname in uploaded.keys():
        src = Path('/content') / fname
        dst = ONTO / fname
        shutil.move(str(src), str(dst))
    print("Uploaded to:", ONTO)
else:
    print("All required ontology files are present in:", ONTO)


## 3) Robust parsing with rdflib

`.xrdf` is not a standard extension, so we try common RDF/OWL formats.


In [ ]:
from rdflib import Graph

def load_graph(path: Path):
    g = Graph()
    tried = []
    for fmt in ["xml", "application/rdf+xml", "turtle", "n3", "nt"]:
        try:
            g.parse(str(path), format=fmt)
            return g, fmt, tried
        except Exception as e:
            tried.append((fmt, str(e)[:200]))
            continue
    # last attempt: autodetect
    g.parse(str(path))
    return g, "auto", tried

pdon_path = ONTO / "pdon.xrdf"
pmdo_path = ONTO / "pmdo.xrdf"

pdon_g, pdon_fmt, pdon_tried = load_graph(pdon_path)
pmdo_g, pmdo_fmt, pmdo_tried = load_graph(pmdo_path)

print("PDON triples:", len(pdon_g), "format:", pdon_fmt)
print("PMDO triples:", len(pmdo_g), "format:", pmdo_fmt)


## 4) Extract basic ontology statistics

We count OWL classes and properties, and extract ontology IRI(s) if present.


In [ ]:
import pandas as pd
from rdflib.namespace import RDF, RDFS, OWL

def ontology_stats(g: Graph, name: str):
    ont_iris = list(g.subjects(RDF.type, OWL.Ontology))
    classes  = set(g.subjects(RDF.type, OWL.Class))
    objp     = set(g.subjects(RDF.type, OWL.ObjectProperty))
    datap    = set(g.subjects(RDF.type, OWL.DatatypeProperty))
    annp     = set(g.subjects(RDF.type, OWL.AnnotationProperty))
    return {
        "ontology": name,
        "triples": len(g),
        "ontology_iris": ";".join(str(x) for x in ont_iris[:10]),
        "n_classes": len(classes),
        "n_object_properties": len(objp),
        "n_datatype_properties": len(datap),
        "n_annotation_properties": len(annp),
    }

stats = pd.DataFrame([
    ontology_stats(pdon_g, "PDON"),
    ontology_stats(pmdo_g, "PMDO"),
])

stats


## 5) Save summary to CSV

In [ ]:
summary_csv = OUT / "ontologies_summary.csv"
stats.to_csv(summary_csv, index=False)
summary_csv


## 6) Quick label search helpers

Useful for locating candidate classes/properties by `rdfs:label`.


In [ ]:
def find_by_label(g: Graph, query: str, limit: int = 25):
    q = query.lower()
    hits = []
    for s, _, o in g.triples((None, RDFS.label, None)):
        if q in str(o).lower():
            hits.append((str(s), str(o)))
            if len(hits) >= limit:
                break
    return pd.DataFrame(hits, columns=["iri", "label"])

# Example searches (edit freely)
find_by_label(pdon_g, "parkinson", limit=10)


## 7) Optional: export Turtle versions

This can make later processing easier. Turtle files are written to `output/`.


In [ ]:
pdon_ttl = OUT / "pdon.ttl"
pmdo_ttl = OUT / "pmdo.ttl"

pdon_g.serialize(destination=str(pdon_ttl), format="turtle")
pmdo_g.serialize(destination=str(pmdo_ttl), format="turtle")

pdon_ttl, pmdo_ttl


## 8) Next notebook

Proceed to `02_build_bridge_ontology.ipynb` to create the **bridge ontology** with:
- `owl:imports` to PDON and PMDO
- minimal T-box for Subject/Visit/Observation
- mapping scaffolding for PPMI variables
